In [ ]:
import h5py
import numpy as np
from astropy.table import Table, vstack
import pandas as pd
import matplotlib.pyplot as plt
from astropy.table import Table
import numpy as np
import math
import warnings
import matplotlib.pyplot as plt

def read_and_process_fits(files):
    dat = None
    for muon_file in files:
        dat2 = Table.read(muon_file, format='fits')
        
        # Convert columns to boolean if they exist
        for col in ['good_ring', 'is_valid', 'parameters_at_limit']:
            if col in dat2.colnames:
                dat2[col] = dat2[col].astype(bool)
        
        # Stack the tables
        dat = vstack([dat, dat2]) if dat is not None else dat2
    
    return dat

ctapipe_dl1_file_path = "/Users/vdk/Software/ctapipe_processor_test/events.dl1.h5"
ctapipe_dl1_file_path = "/Users/vdk/Software/ctapipe_processor_test/test_images/dl1_20k_muons.h5"

# Getting muon table for ctapipe

In [ ]:
with h5py.File(ctapipe_dl1_file_path, "r") as h5_file:
    dataset = h5_file["dl1/event/telescope/muon/tel_001"]
    dataset_dtype = dataset.dtype
    data = dataset[:]

data_array = np.array(data, dtype=dataset_dtype)
muon_table_ctapipe = Table(data_array)
#muon_table_ctapipe[muon_table_ctapipe['muonefficiency_is_valid'] == 1]
muon_table_ctapipe[muon_table_ctapipe['muonefficiency_optical_efficiency'] > 0]

In [ ]:
muon_table_lstchain = read_and_process_fits(["/Users/vdk/Software/ctapipe_processor_test/lstchain_comparison/20k_events/muon_table.fits"])

muon_table_lstchain[muon_table_lstchain['good_ring'] == True]

In [ ]:
np.nanmean(muon_table_ctapipe['muonparameters_ring_size'])

In [ ]:
np.nanmean(muon_table_lstchain['ring_size'])

In [ ]:
comparison_columns_lstchain = [
    'ring_size', 
    'ring_center_x', 
    'ring_center_y', 
    'ring_radius',  
    'ring_width', 
    'impact_parameter', 
    'muon_efficiency',
    'radial_stdev',
    ]

for value in muon_table_lstchain[muon_table_lstchain['event_id'] == 303]:
    for col in value.colnames:
        if col in comparison_columns_lstchain:
            print(f"{col}: {value[col]}")

In [ ]:
comparison_columns_ctapipe = [
    'muonparameters_ring_size', 
    'muonring_center_fov_lon', 
    'muonring_center_fov_lat', 
    'muonring_radius',  
    'muonefficiency_width', 
    'muonefficiency_impact',
    'muonefficiency_optical_efficiency',
    'muonparameters_standard_dev'
]
for value in muon_table_ctapipe[muon_table_ctapipe['event_id'] == 303]:
    for col in value.colnames:
        if col in comparison_columns_ctapipe:
            print(f"{col}: {value[col]}")

# Plot comparison

In [ ]:
def reweight_gaug(data, zenith, alpha  = 0.7, is_data=False):
    if is_data:
        weights = np.ones(len(data))/len(data)
    else:
        zenith = np.deg2rad(zenith)
        weights = np.ones(len(data))
        weights = data['mc_energy']**(-alpha) * (1/(1 + 1.1 * data['mc_energy'] * 1000 * np.cos(np.deg2rad(zenith))/115) + 0.054/(1 + 1.1 * data['mc_energy'] * 1000 * np.cos(np.deg2rad(zenith))/850))
    return weights / np.sum(weights)
    
def read_and_process_fits(files):
    dat = None
    for muon_file in files:
        dat2 = Table.read(muon_file, format='fits')
        
        # Convert columns to boolean if they exist
        for col in ['good_ring', 'is_valid', 'parameters_at_limit']:
            if col in dat2.colnames:
                dat2[col] = dat2[col].astype(bool)
        
        # Stack the tables
        dat = vstack([dat, dat2]) if dat is not None else dat2
    
    return dat

def plot_columns(df_lstchain, df_ctapipe, columns_lstchain, columns_ctapipe, rows, cols, bin_power = 0.4, apply_weight=True, plot_option=True):
    fig, axes = plt.subplots(rows, cols, figsize=(12, 18))
    axes = axes.flatten()
    
    for i, col in enumerate(columns_lstchain):
        lstchain_values = df_lstchain[col][~np.isnan(df_lstchain[col])]
        ctapipe_values = df_ctapipe[columns_ctapipe[i]][~np.isnan(df_ctapipe[columns_ctapipe[i]])]
    

        min_value = min(min(ctapipe_values), min(lstchain_values))
        max_value = max(max(ctapipe_values), max(lstchain_values))

        num_bins = int(max(len(df_lstchain)**bin_power, len(df_ctapipe))**bin_power)
        #num_bins = max(10, min(num_bins, 50))

        
        bins = np.linspace(min_value, max_value, num_bins + 1)

        if apply_weight:
            lst_weights = reweight_gaug(df_lstchain, 10, 0.7, False)
            ctapipe_weights = reweight_gaug(df_ctapipe, 10, 0.7, False)
            axes[i].hist(lstchain_values, weights=lst_weights, bins=bins, alpha=0.6, color='k', density=True, label='Data', histtype='step', lw=2)
            axes[i].hist(ctapipe_values, weights=ctapipe_weights, bins=bins, alpha=0.85, color='orange', density=True, label=f'Muon Simulation 0.0046deg', histtype='step', lw=2)
        else:
            axes[i].hist(lstchain_values, bins=bins, alpha=0.6, color='k', density=True, label='lstchain', histtype='step', lw=2)
            axes[i].hist(ctapipe_values, bins=bins, alpha=0.85, color='orange', density=True, label=f'ctapipe', histtype='step', lw=2)

        axes[i].set_title(f'Histogram of {col}')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('Normalized Frequency')
        axes[i].legend(loc='upper right')

    plt.tight_layout()
    if plot_option:
        plt.savefig(f'/Users/vdk/muons2024/basic_distributions/PROTONS_noringcenter_{name}_radiuscut={min_radius_cut}-{max_radius_cut}_completeness={ring_completeness_cut}_centerdistance={ring_center_distance_cut}_impactdistance={impact_distance_cut}_widthquantile={QUANTILE_CUT}.pdf', dpi=200)
    plt.show()

comparison_columns_lstchain = [
    'ring_size', 
    'ring_center_x', 
    'ring_center_y', 
    'ring_radius',  
    'ring_width', 
    'impact_parameter', 
    'muon_efficiency',
    'radial_stdev',
    ]
comparison_columns_ctapipe = [
    'muonparameters_ring_size', 
    'muonring_center_fov_lon', 
    'muonring_center_fov_lat', 
    'muonring_radius',  
    'muonefficiency_width', 
    'muonefficiency_impact',
    'muonefficiency_optical_efficiency',
    'muonparameters_standard_dev'
]

In [ ]:
plot_columns(
    #muon_table_lstchain[muon_table_lstchain['muon_efficiency'] < ], 
    #muon_table_ctapipe[muon_table_ctapipe['muonefficiency_optical_efficiency'] < 1], 
    muon_table_lstchain,
    muon_table_ctapipe,
    comparison_columns_lstchain, 
    comparison_columns_ctapipe, 
    rows=4,
    cols=2,
    apply_weight=False, 
    plot_option=False, 
)

In [ ]:
plot_columns(
    muon_table_lstchain[muon_table_lstchain['muon_efficiency'] < 1], 
    muon_table_ctapipe[muon_table_ctapipe['muonefficiency_optical_efficiency'] < 1], 
    comparison_columns_lstchain, 
    comparison_columns_ctapipe,
    rows=4,
    cols=2,
    bin_power=0.6,
    apply_weight=False, 
    plot_option=False, 
)

In [ ]:
plt.figure(figsize=(10, 7))
z,x,c = plt.hist(muon_table_lstchain['ring_radius'], density=True, bins=50, alpha=0.5, label='lstchain')
z,x,c = plt.hist(muon_table_ctapipe['muonring_radius'], density=True, bins=50, alpha=0.5, label='ctapipe')
plt.legend()
plt.xlabel('Ring radius')

In [ ]:
plt.figure(figsize=(10, 7))
z,x,c = plt.hist(muon_table_lstchain['ring_center_y'], density=True, bins=np.linspace(-1,1,40), alpha=0.5, label='lstchain')
z,x,c = plt.hist(muon_table_ctapipe['muonring_center_fov_lat'], density=True, bins=np.linspace(-1,1,40), alpha=0.5, label='ctapipe')
plt.legend()
plt.xlabel('Ring radius')

In [ ]:
plt.figure(figsize=(10, 7))
z,x,c = plt.hist(muon_table_lstchain['ring_center_x'], density=True, bins=np.linspace(-1,1,40), alpha=0.5, label='lstchain')
z,x,c = plt.hist(muon_table_ctapipe['muonring_center_fov_lon'], density=True, bins=np.linspace(-1,1,40), alpha=0.5, label='ctapipe')
plt.legend()
plt.xlabel('Ring radius')

In [ ]:
np.isnan(muon_table_ctapipe['muonefficiency_impact'][0])

In [ ]:
len(muon_table_ctapipe[np.isnan(muon_table_ctapipe['muonefficiency_impact'])])

In [ ]:
compare_astropy_tables_overlaid_histograms(
    muon_table_ctapipe=muon_table_ctapipe,
    muon_table_lstchain=muon_table_lstchain,
    comparison_columns_ctapipe=comparison_columns_ctapipe,
    comparison_columns_lstchain=comparison_columns_lstchain,
)